<a href="https://colab.research.google.com/github/FXtest2023/FXtest2023/blob/main/1004_USDJPY_PCA_maxdepth5_save3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install pandas_ta
!pip install optuna

In [18]:

from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as pd_ta
from sklearn.model_selection import KFold
from optuna.integration import lightgbm as lgb
#import lightgbm as lgb
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import numpy as np
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
import random

In [19]:
url = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/USDJPY_1D.csv"
df_price = pd.read_csv(url)

In [20]:
df_price['diff']  = df_price['bid_close'].shift(-1) - df_price['bid_close']
df_price['label'] = df_price['diff'].apply(lambda x: 1 if x>0 else 0 )
df_price.dropna(inplace=True)
df_price.reset_index(drop=True,inplace=True)

In [21]:
JPYnews_url = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/JP_market_news_1D.csv"
df_JPYnews =  pd.read_csv(JPYnews_url)

In [22]:
USDnews_url = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/US_market_news_1D.csv"
df_USDnews =  pd.read_csv(USDnews_url)

In [23]:
JPY_currency_news_url = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/JPY_currency_news_1D.csv"
df_JPY_currency_news =  pd.read_csv(JPY_currency_news_url)

In [24]:
USD_currency_news_url = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/USD_currency_news_1D.csv"
df_USD_currency_news =  pd.read_csv(USD_currency_news_url)

In [25]:
USD_PCA_url = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/PCA_US_v1.csv"
df_USD_PCA =  pd.read_csv(USD_PCA_url)

In [26]:
JPY_PCA_url = "https://raw.githubusercontent.com/FXtest2023/FXtest2023/main/PCA_JP_v1.csv"
df_JPY_PCA =  pd.read_csv(JPY_PCA_url)

In [27]:
# df_1H = df_price.merge(df_USDnews, how='inner', on='utc_datetime')
# df_1H = df_1H.merge(df_JPYnews, how='inner', on='utc_datetime')
# df_1H = df_1H.merge(df_JPY_currency_news, how='inner', on='utc_datetime')
# df_1H = df_1H.merge(df_USD_currency_news, how='inner', on='utc_datetime')
df_1H = df_price.merge(df_USD_PCA, how='inner', on='utc_datetime')
df_1H = df_1H.merge(df_JPY_PCA, how='inner', on='utc_datetime')

In [28]:
# df_1H = df_1H.loc[df_1H.utc_datetime >'2019']
df_1H

,utc_datetime,bid_open,bid_high,bid_low,bid_close,diff,label,US_stockindex_1,US_stockindex_2,US_bond_1,...,JP_bond_3,JP_rates_1,JP_rates_2,JP_rates_3,JP_currency_1,JP_currency_2,JP_currency_3,JP_currency_4,JP_currency_5,JP_currency_6
0,2019-01-02 00:00:00+00:00,109.651,109.714,109.596,109.657,-2.243,0,0.048647,-0.021033,-0.096121,...,-0.026600,1.155146,-0.309756,-0.069851,-0.113325,0.155290,0.009371,-0.019068,0.043101,-0.057756
1,2019-01-03 00:00:00+00:00,109.656,109.666,105.247,107.414,0.422,1,0.167171,-0.005396,0.053550,...,-0.020142,0.128112,0.158260,0.008852,0.136855,0.061961,0.076961,0.007007,0.000951,-0.012460
2,2019-01-04 00:00:00+00:00,107.414,108.294,106.728,107.836,0.598,1,0.139192,0.024593,0.064574,...,0.005928,0.019869,0.090450,0.087531,0.085441,0.043240,0.059947,0.027737,-0.019258,0.022617
3,2019-01-05 00:00:00+00:00,107.836,108.573,107.500,108.434,0.100,1,-0.060074,0.007104,-0.044305,...,-0.026503,-0.129017,-0.061540,-0.007567,0.056212,0.069227,0.036923,0.002217,-0.017046,0.025397
4,2019-01-07 00:00:00+00:00,108.508,108.609,108.338,108.534,0.078,1,-0.057869,0.012932,-0.031120,...,-0.029418,0.321081,-0.045789,-0.021378,0.158491,-0.054932,0.069958,-0.005412,-0.020448,0.073625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,2023-08-26 00:00:00+00:00,146.029,146.609,145.704,146.353,0.171,1,0.066148,-0.020096,0.043295,...,0.049328,-0.050318,0.018048,0.013352,0.019827,-0.141827,-0.036371,0.011308,0.016900,-0.009093
1453,2023-08-28 00:00:00+00:00,146.393,146.591,146.377,146.524,-0.085,0,-0.041599,0.012007,-0.003133,...,-0.069717,-0.029113,0.010165,-0.004628,-0.157032,-0.037979,-0.024075,0.023452,0.008373,0.000050
1454,2023-08-29 00:00:00+00:00,146.523,146.722,146.250,146.439,-0.477,0,-0.118344,-0.022286,0.026090,...,-0.030933,-0.020920,0.052492,-0.005086,-0.167888,-0.038968,-0.043201,-0.015777,-0.013367,-0.027423
1455,2023-08-30 00:00:00+00:00,146.438,147.354,145.646,145.962,0.102,1,-0.076214,-0.013886,0.021250,...,-0.028160,0.032732,0.024311,0.029853,0.018060,-0.132834,-0.040798,0.028361,-0.037806,-0.033464


In [29]:

print(list(df_1H.columns))

['utc_datetime', 'bid_open', 'bid_high', 'bid_low', 'bid_close', 'diff', 'label', 'US_stockindex_1', 'US_stockindex_2', 'US_bond_1', 'US_bond_2', 'US_bond_3', 'US_rates_1', 'US_rates_2', 'US_currency_1', 'US_currency_2', 'US_currency_3', 'US_currency_4', 'US_currency_5', 'US_currency_6', 'JP_stockindex_1', 'JP_stockindex_2', 'JP_bond_1', 'JP_bond_2', 'JP_bond_3', 'JP_rates_1', 'JP_rates_2', 'JP_rates_3', 'JP_currency_1', 'JP_currency_2', 'JP_currency_3', 'JP_currency_4', 'JP_currency_5', 'JP_currency_6']


In [30]:
# 乱数シードの固定
SEED = 42

# LightGBM訓練用関数を定義
def trainGbdt(X_train, X_val, y_train, y_val):

    #LGB用のデータに変形
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)

# need to set deterministic and force_col_wise
# https://github.com/microsoft/LightGBM/issues/3761
    param = {
            'objective': 'binary',
            'metric': 'auc',
            'boosting_type': 'gbdt',
            'random_seed':SEED,
            'deterministic': True,
            'force_col_wise': True,
            'max_depth':5,
            'verbose':-1,
            'min_data_in_leaf': 60
#            'device': 'gpu',
 #           'force_col_wise': True,
        }

        # 訓練実施
    model = lgb.train(param,
                    lgb_train,
                    valid_sets=[lgb_train,lgb_val],
                    valid_names=('train', 'val'),
#                    callbacks=([lgb.early_stopping(stopping_rounds=100)]),
                    verbose_eval=False
                    )
    print(model.params, model.best_iteration, model.best_score)

    return model

In [ ]:
df_1H_cols = list(df_1H.columns)
no_feature_cols = ['utc_datetime','bid_open',	'bid_high',	'bid_low','bid_close', 'diff','label']
result = pd.DataFrame(columns = ['feature','number of features', 'train acc','validation acc','test acc','F score','random seed','features', 'best_feature'])
test_cols = {"stockindex": ['US_stockindex_1', 'US_stockindex_2','JP_stockindex_1', 'JP_stockindex_2'],
             "bond": ['US_bond_1', 'US_bond_2', 'US_bond_3','JP_bond_1', 'JP_bond_2', 'JP_bond_3'],
             "rates": ['US_rates_1', 'US_rates_2','JP_rates_1', 'JP_rates_2', 'JP_rates_3'],
             "currency": ['US_currency_1', 'US_currency_2', 'US_currency_3', 'US_currency_4', 'US_currency_5', 'US_currency_6',
                          'JP_currency_1', 'JP_currency_2', 'JP_currency_3', 'JP_currency_4', 'JP_currency_5', 'JP_currency_6'],
             "stockindex&bond": ['US_stockindex_1', 'US_stockindex_2','JP_stockindex_1', 'JP_stockindex_2','US_bond_1', 'US_bond_2', 'US_bond_3','JP_bond_1', 'JP_bond_2', 'JP_bond_3'],
             "stockindex&rates": ['US_stockindex_1', 'US_stockindex_2','JP_stockindex_1', 'JP_stockindex_2','US_rates_1', 'US_rates_2','JP_rates_1', 'JP_rates_2', 'JP_rates_3'],
             "bond&rates": ['US_bond_1', 'US_bond_2', 'US_bond_3','JP_bond_1', 'JP_bond_2', 'JP_bond_3','US_rates_1', 'US_rates_2','JP_rates_1', 'JP_rates_2', 'JP_rates_3']}
for k, v in test_cols.items():
    feature_cols = v
    cols = no_feature_cols + feature_cols

    df = df_1H[cols]
    df_train = df[ df['utc_datetime']<'2022-07']
    df_val = df[(df['utc_datetime']>'2022-07') & (df['utc_datetime']<'2023')]

    df_test = df[(df['utc_datetime']>'2023') & (df['utc_datetime']<'2023-09')]
    X_train=df_train.drop(columns=no_feature_cols)
    y_train=df_train['label']

    X_val=df_val.drop(columns=no_feature_cols)
    y_val=df_val['label']


    X_test=df_test.drop(columns=no_feature_cols)
    y_test=df_test['label']

    model_result = trainGbdt(X_train, X_val, y_train, y_val)

    y_pred_train = model_result.predict(X_train, num_iteration=model_result.best_iteration)
    y_pred_val = model_result.predict(X_val, num_iteration=model_result.best_iteration)
    y_pred_test = model_result.predict(X_test, num_iteration=model_result.best_iteration)

    new_row = {'feature': k,
               'number of features': len(v),
               'train acc': accuracy_score(y_train, np.round(y_pred_train)),
               'validation acc': accuracy_score(y_val, np.round(y_pred_val)),
               'test acc': accuracy_score(y_test, np.round(y_pred_test)),
               'F score': f1_score(y_test, np.round(y_pred_test)),
               'random seed': SEED,
               'features': feature_cols,
               'best_feature': feature_cols[np.argmax(model_result.feature_importance(importance_type='gain'))]}
    result.loc[len(result)] = new_row

result.to_csv('USDJPY_PCA_news_test.csv', index=False, encoding = 'utf-8-sig')
files.download('USDJPY_PCA_news_test.csv')



[I 2023-10-04 14:36:56,204] A new study created in memory with name: no-name-24685303-339c-4bee-bdd9-c0d79767472f
bagging, val_score: 0.548553:  90%|######### | 9/10 [00:05<00:00,  2.06it/s]